# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

Посмотрим на данные в выборках

In [2]:
dt1 = pd.read_csv('/datasets/geo_data_0.csv')
dt2 = pd.read_csv('/datasets/geo_data_1.csv')
dt3 = pd.read_csv('/datasets/geo_data_2.csv')

dt_list = [dt1, dt2, dt3]
name_list = ['Регион 1', 'Регион 2', 'Регион 3']

for dt, name in zip(dt_list, name_list):
    print(name)
    display(dt.head())
    dt.info()
    print('Количество дубликатов:', dt.duplicated().sum())
    display(dt.isna().sum())
    print('**********************************')
    print()

Регион 1


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
Количество дубликатов: 0


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

**********************************

Регион 2


...

**********************************

Регион 3


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
Количество дубликатов: 0


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

**********************************



Дубликатов не наблюдается, пропусков в данных также нет. Типы значений корректные

Проверим признаки в выборках на мультиколлениарность. Для определения связей построим матрицу парных коэффициентов корреляции.

In [3]:
for name, dt in zip(name_list, dt_list):
    print(name)
    display(dt[['f0', 'f1', 'f2']].corr())
    print('*****************************')
    print()

Регион 1


,f0,f1,f2
f0,1.000000,-0.440723,-0.003153
f1,-0.440723,1.000000,0.001724
f2,-0.003153,0.001724,1.000000


*****************************

Регион 2


,f0,f1,f2
f0,1.000000,0.182287,-0.001777
f1,0.182287,1.000000,-0.002595
f2,-0.001777,-0.002595,1.000000


*****************************

Регион 3


,f0,f1,f2
f0,1.000000,0.000528,-0.000448
f1,0.000528,1.000000,0.000779
f2,-0.000448,0.000779,1.000000


*****************************



In [4]:
display(dt2.corr())

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


Сильных связей между признаками не наблюдается.  

Cтолбцы id содержат данные которые не имеют значеняи для предсказания объёмов запасов нефти в скважинах. Удалим эти столбцы.

In [5]:
for i in range(len(dt_list)):
    dt_list[i] = dt_list[i].drop('id', axis=1)
    print(name_list[i])
    display(dt_list[i].head(2))
    print('**********************************')
    print()

Регион 1


,f0,f1,f2,product
0,0.705745,-0.497823,1.22117,105.280062
1,1.334711,-0.340164,4.36508,73.037750


**********************************

Регион 2


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261


**********************************

Регион 3


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697


**********************************



Данные подготовили.

## Обучение и проверка модели

Перейдем к обучению и проверке модели.  Целевой признак у нас количественный, решается вопрос регрессии. По условиям задачи для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые). Создадим функцию принимающую данные по  региону и выполняющую:
1. Разделение данных по региону на обучающую и валидационную, в соотношении 75:25
2. Обучение модели линейной регрессии
3. Предсказание на валидационных данных
4. Расчет и печать среднего предсказанного запаса сырья
5. Расчет и печать значения RMSE модели
6. Проверку модели на адекватность, для этого сравним полученные предсказания со средним значением целевого признака в валидационной выборке
7. Функция возвращает предсказания и правильные ответы на валидационной выборке


In [6]:
def train_test_model(data):
    target = data['product']
    features = data.drop('product', axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(
                                            features, target, test_size=0.25, random_state=12345)
    print('Размер обучающей выборки:', features_train.shape)
    print('Размер валидационной выборки:', features_valid.shape)
        
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    
    predictions_series = pd.Series(predictions)
    
    average_reserve = predictions.mean()
    rmse = (mean_squared_error(target_valid, predictions_series))**(0.5)
    
    print("Средний предсказанный запас сырья: {0:.3f} тыс. баррелей".format(average_reserve))
    print("RMSE: {0:.2f}".format(rmse))
    
    #проверка на адекватность
    predictions_for_mean = pd.Series(target_valid.mean(), index=target_valid.index)
    rmse_for_mean = (mean_squared_error(target_valid, predictions_for_mean))**(0.5)
    print("RMSE для для среднего значения: {0:.2f}".format(rmse_for_mean))
    
    return (predictions_series.reset_index(drop=True), target_valid.reset_index(drop=True))
    

Подготовим словарь для хранения предсказаний и правильных ответов из валидационной выборки для всех регионов

In [7]:
region_data_dict = {}
list_name_values = ["predictions", "target_valid"]
for name in name_list:
    dict_temp = {}
    for name_value in list_name_values:
        dict_temp[name_value] = None
    region_data_dict[name] =  dict_temp  

#проверим
display(region_data_dict)

{'Регион 1': {'predictions': None, 'target_valid': None},
 'Регион 2': {'predictions': None, 'target_valid': None},
 'Регион 3': {'predictions': None, 'target_valid': None}}

Выполним обучение и проверку моделей для всех регионов.

In [8]:
for dt, name in zip(dt_list, name_list):
    print(name)
    print()
    (region_data_dict[name]['predictions'], 
         region_data_dict[name]['target_valid']) = train_test_model(dt)
    print('*************************************')
    print()

Регион 1

Размер обучающей выборки: (75000, 3)
Размер валидационной выборки: (25000, 3)
Средний предсказанный запас сырья: 92.593 тыс. баррелей
RMSE: 37.58
RMSE для для среднего значения: 44.29
*************************************

Регион 2

Размер обучающей выборки: (75000, 3)
Размер валидационной выборки: (25000, 3)
Средний предсказанный запас сырья: 68.729 тыс. баррелей
RMSE: 0.89
RMSE для для среднего значения: 46.02
*************************************

Регион 3

Размер обучающей выборки: (75000, 3)
Размер валидационной выборки: (25000, 3)
Средний предсказанный запас сырья: 94.965 тыс. баррелей
RMSE: 40.03
RMSE для для среднего значения: 44.90
*************************************



Модель адекватна, на всех выборках точность предсказания выше чем для среднего значения.  
Самый высокий предсказанный средний запас нефти в 3-м регионе, при этом здесь самое высокое значение ошибки. Во 2-м регионе ошибка низкая, но запас предсказан меньше других.   

## Подготовка к расчёту прибыли

Перейдем к рассчету прибыли. Все ключевые значения для расчётов сохраним в отдельных переменных.

In [9]:
BUDGET = 10000000000  #бюджет на разработку скважин в регионе
INCOME_UNIT = 450000  #доход с каждой тысячи баррелей
WELLS_RESEARCH = 500  #количество точек (размер выборки) для исследования в регионе
WELLS_SELECT = 200    #количество лучших точек
SAMPLES_COUNT = 1000  #количество выборо для Bootstrap
STATE = np.random.RandomState(12345)

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [10]:
capacity_product_min = BUDGET / INCOME_UNIT / WELLS_SELECT
print('Объём сырья для безубыточной разработки новой скважины: {0:.3f} тыс. баррелей'.format(capacity_product_min))

Объём сырья для безубыточной разработки новой скважины: 111.111 тыс. баррелей


Средний предсказанный запас сырья в скважинах во всех регионах меньше, чем рассчитанный для данного бюджета объём сырья для безубыточной разработки. 

Создадим функцию для рассчета прибыли. Функция принимает предсказания и верные ответы, отбирает WELLS_SELECT лучших точек по объему предсказанных запасов сырья, и рассчитывает по соответствующим целевым признакам уровень дохода.

In [11]:
def profit_calc_old(predictions, target):
    top200_predictions = predictions.sort_values(ascending=False)[:WELLS_SELECT]
    profit = target[top200_predictions.index].sum() * INCOME_UNIT - BUDGET
    return profit

In [12]:
def profit_calc(predictions, target):
    top200_predictions = predictions.sort_values(ascending=False)
    profit = target[top200_predictions.index][:WELLS_SELECT].sum() * INCOME_UNIT - BUDGET   
    return profit

## Расчёт прибыли и рисков 

Посчитаем риски и прибыль для каждого региона. Для этого создадим функцию, где применим технику Bootstrap с SAMPLES_COUNT выборок, чтобы найти распределение прибыли. Рассчитаем среднюю прибыль, 95%-й доверительный интервал и риск убытков (процент отрицательной прибыли).

In [13]:
def profit_risk_calc(predictions, target):
    profits = []
    for i in range(SAMPLES_COUNT):
        predict_subsample = predictions.sample(WELLS_RESEARCH, replace=True, random_state=STATE)
        target_subsample = target[predict_subsample.index]
        profits.append(profit_calc(predict_subsample, target_subsample))
    profits = pd.Series(profits)
    
    mean_profit = profits.mean()
    conf_interval = (profits.quantile(.025), profits.quantile(.975))
    risk = st.percentileofscore(profits, 0)
    
    return (conf_interval, mean_profit, risk)

Применим функцию для рассчета показателей по регионам

In [14]:
for name in name_list:
    print(name)
    print()
    conf_interval, mean_profit, risk = (profit_risk_calc(region_data_dict[name]['predictions'], 
                                                     region_data_dict[name]['target_valid']))
    print('Средняя прибыль:', round(mean_profit, 2))
    print('95%-й доверительный интервал:', conf_interval)
    print('Риск убытков:', risk, '%')
    print('**********************************')
    print()
    

Регион 1

Средняя прибыль: 425938526.91
95%-й доверительный интервал: (-102090094.83793654, 947976353.358369)
Риск убытков: 6.0 %
**********************************

Регион 2

Средняя прибыль: 518259493.7
95%-й доверительный интервал: (128123231.43308629, 953612982.0669085)
Риск убытков: 0.3 %
**********************************

Регион 3

Средняя прибыль: 420194005.34
95%-й доверительный интервал: (-115852609.16001143, 989629939.844574)
Риск убытков: 6.2 %
**********************************



По результатам рассчетов риск убытков только для 2-го региона меньше целевого показателя 2.5%. Из всех регионов наибольшую среднюю прибыль показывает также регион №2. 

## Вывод

Для решения поставленной задачи, были исследованы данные характеристик скважин в 3-х регионах. Проведено обучение и проверка модели линейной регрессии. Модель прошла проверку на адекватность. 
Самый высокий предсказанный средний запас нефти был получен 3-м регионе, при этом для него было и самое высокое значение ошибки(RMSE). Во 2-м регионе значение ошибки было самым низким, но и запас предсказан меньше других.  

Для определения региона, где добыча принесёт наибольшую прибыль была применена техника Bootstrap с 1000 выборок. Были рассчитаны средняя прибыль, 95%-й доверительный интервал и риск убытков. Самым перспективными регионом для разработки определен регион №2, для него получен самый высокий средний показатель прибыли при самом низком риске убытков.